# Change current working directory to wherever the dataset is located

The code in this notebook was used to upscale and center-crop the images and masks in the datasets.
Mark sure to change the working directory to wherever your dataset is currently stored.

In [1]:
import os
from PIL import Image

In [2]:
# Replace the absolute_path with the absolute path you want to set
absolute_path = "/Users/claire/Downloads/DUTS-TR"

# Use os.chdir to change the working directory
os.chdir(absolute_path)

In [3]:
%pwd

'/Users/claire/Downloads/DUTS-TR'

# Code for center cropping images while avoiding distortion

Make sure to change where you want to store the rescaled and center-cropped images

In [6]:
def resize_and_crop(image_path, segmentation_path, output_size, output_image_folder, output_segmentation_folder):
    # Open the image and segmentation map
    image = Image.open(image_path)
    segmentation = Image.open(segmentation_path)

    # Calculate the aspect ratio
    aspect_ratio = image.width / image.height

    # Determine the new size for resizing
    if aspect_ratio < 1:
        new_width = output_size
        new_height = int(output_size / aspect_ratio)
    else:
        new_height = output_size
        new_width = int(output_size * aspect_ratio)

    # Resize the image and segmentation map
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    resized_segmentation = segmentation.resize((new_width, new_height), Image.Resampling.NEAREST)

    # Calculate the center crop box
    left = (new_width - output_size) / 2
    top = (new_height - output_size) / 2
    right = (new_width + output_size) / 2
    bottom = (new_height + output_size) / 2

    # Perform the center crop
    cropped_image = resized_image.crop((left, top, right, bottom))
    cropped_segmentation = resized_segmentation.crop((left, top, right, bottom))

    # Save the results
    image_filename = os.path.basename(image_path)
    segmentation_filename = os.path.basename(segmentation_path)

    output_image_path = os.path.join(output_image_folder, image_filename)
    output_segmentation_path = os.path.join(output_segmentation_folder, segmentation_filename)

    cropped_image.save(output_image_path)
    cropped_segmentation.save(output_segmentation_path)

def process_images(input_image_folder, input_segmentation_folder, output_image_folder, output_segmentation_folder, output_size):
    # Ensure the output folders exist
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_segmentation_folder, exist_ok=True)

    # Process each image in the input folders
    for filename in os.listdir(input_image_folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(input_image_folder, filename)
            segmentation_path = os.path.join(input_segmentation_folder, filename[:-4] + ".png")

            # Resize and crop the image and segmentation map
            resize_and_crop(image_path, segmentation_path, output_size, output_image_folder, output_segmentation_folder)

if __name__ == "__main__":
    input_image_folder = "img"
    input_segmentation_folder = "gt"
    output_image_folder = "resized_img"
    output_segmentation_folder = "resized_gt"
    output_size = 512

    process_images(input_image_folder, input_segmentation_folder, output_image_folder, output_segmentation_folder, output_size)
